Author: Vinícius Antoino Ramos Zecca

*LinkedIn: /viniciusarz*

*Github: /ViniciusARZ*


In [3]:
import sqlite3
import pandas as pd
from IPython.display import display
import sqlalchemy


# Data Cleaning and Processing (w/ Tools)

## Google Spreadsheets

In Google Spreadsheets I cleaned up what it was faster than it would be using SQL:
1. Replace "N/A's with " ", or basically *nulls*
2. Transform some Attribute types to make it easier to read in SQL.
3. Column "Faturamento" was in Text but it actually is a currency, so I used SPLIT() + TRIM() + SUBSTITUTE() + 'Number Formatting' to correct it... Same thing  with others.
4. Select which Columns I'm going to need based on the problem I want to solve and the analysis object. Did a backup in case I wanted to return. 


## SQL - Google Cloud's Big Query

Disclaimers:
1. I didn't use SQLite extension because I already had everything in Big Query
2. The following Query Codes are just to show what I did in SQL and keep a Changelog




Import and Schema Establishment

In [19]:
#'ID:INTEGER,EMPRESA_JUNIOR:STRING,DATA_DE_CADASTRO:DATE,ANO_DE_FEDERACAO:INTEGER, CLUSTER_2020:INTEGER,CLUSTER_2021:INTEGER,CLUSTER_2022:INTEGER,É_ALTO_CRESCIMENTO:BOOLEAN, MES_DO_ALTO_CRESCIMENTO:INTEGER,TEMPO_MEDIO_POR_PROJETO_DIAS:FLOAT,PORC_PROJETOS_COLETADOS_NPS:INTEGER,DATA_DE_ASSINATURA:DATE,MES:INTEGER,PROJETOS:INTEGER,DURACAO_DO_PROJETO_EM_DIAS_UTEIS:INTEGER,FATURAMENTO:FLOAT,ATINGE_ALGUM_ODS:BOOLEAN,NUMERO_DE_ODS_ATINGIDOS:INTEGER,ACAO_COMPARTILHADA:BOOLEAN,NUMERO_DE_EJS_QUE_PARTICIPARAM:INTEGER,NUMERO_DE_MEMBROS_PARTICIPANTES:INTEGER,NPS:INTEGER,DATA_DE_COLETA_DE_NPS:DATE,ESTADO_DO_CLIENTE:STRING,CIDADE_DO_CLIENTE:STRING,SERVICO:STRING,ESTADO:STRING,CIDADE:STRING,UNIVERSIDADE:STRING,CURSOS:STRING,NUMERO_MEMBROS:INTEGER'

# Dataframe reading just to visualize the table
df2021 = pd.read_csv('/content/BJ_NuSC_Mon2021_07 - 2021.csv',',')
display(df2021)


,ID,EMPRESA JÚNIOR,DATA DE CADASTRO,ANO DE FEDERAÇÃO,CLUSTER 2020,CLUSTER 2021,CLUSTER 2022,É DE ALTO CRESCIMENTO?,MÊS DO ALTO CRESCIMENTO,TEMPO MÉDIO POR PROJETO (DIAS),% DE PROJETOS COLETADOS NPS,DATA DE ASSINATURA,MÊS,PROJETOS,DURAÇÃO DO PROJETO (EM DIAS ÚTEIS),FATURAMENTO,ATINGE ALGUM ODS?,Nº DE ODS ATINGIDOS,AÇÃO COMPARTILHADA?,Nº DE EJS QUE PARTICIPARAM,Nº DE MEMBROS PARTICIPANTES,NPS,DATA DE COLETA DE NPS,ESTADO DO CLIENTE,CIDADE DO CLIENTE,SERVIÇO,ESTADO,CIDADE,UNIVERSIDADE,CURSOS,MEMBROS
0,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,01/10/2019,10,2,367,"990,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Desenvolvimento de Aplicativo,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
1,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,02/03/2020,3,1,240,"600,000.00",Sim,3,Não,NaN,2.0,NaN,NaN,SP,São Carlos,Design de layout,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
2,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,19/07/2020,7,1,160,"483,000.00",Sim,1,Não,NaN,1.0,NaN,NaN,AM,Manaus,Design Ambiental,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
3,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,24/07/2020,7,1,116,"400,000.00",Sim,2,Não,NaN,3.0,9.0,30/10/2020,MS,Campo Grande,gestão de resíduos Sólidos,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
4,100,EESC jr.,09/09/2016,1992,5.0,5.0,NaN,Não,NaN,38.63,52.0,09/09/2020,9,1,86,"1,393,758.00",Sim,2,Não,NaN,1.0,10.0,20/01/2021,SP,Santa Bárbara D'Oeste,Design de layout,SP,São Carlos,UNIVERSIDADE DE SÃO PAULO,"ENGENHARIA AERONÁUTICA, ABI - ENGENHARIA MECÂN...",69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
441,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
442,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14
443,2904,EESP Jr.,29/01/2021,2021,NaN,NaN,NaN,Não,NaN,2.41,62.0,19/05/2021,5,1,3,"2,000.00",Sim,2,Não,NaN,1.0,NaN,NaN,SP,São Carlos,Palestras e Workshops,SP,São Carlos,UNIVERSIDADE FEDERAL DE SÃO CARLOS,"EDUCAÇÃO ESPECIAL, EDUCAÇÃO ESPECIAL",14


*The SQL Queries to create Variables are in the Analysis Section below*

# Analysis Phase

In [20]:
# Aggregate data by Junior Enterprise and tell me the Cluster progress rate,
#   so this means that I want to see the proportion between Junior Enterprises that
#   were able to progress from 2020 to 2021 in Cluster classification.
#   QUERY 1
#   SELECT EMPRESA_J__NIOR,
    #   AVG(CLUSTER_2020) AS CLUSTER2020,
    #   AVG(CLUSTER_2021)AS CLUSTER2021,
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY EMPRESA_J__NIOR
    #   ORDER BY C2021 DESC
resultsQuery1 = pd.read_csv('results-query1.csv')
display(resultsQuery1)

,ID,CLUSTER2020,CLUSTER2021
0,100,5.0,5.0
1,163,4.0,5.0
2,421,3.0,5.0
3,137,4.0,5.0
4,400,4.0,4.0
5,302,1.0,4.0
6,131,3.0,4.0
7,142,4.0,4.0
8,168,3.0,4.0
9,1444,1.0,3.0


What can be seen from here?

Taking a Look at Cluster Evolution

In [21]:
# - In SQL:
#   SELECT EMPRESA_J__NIOR,
    #   AVG(CLUSTER_2020) AS CLUSTER2020,
    #   AVG(CLUSTER_2021)AS CLUSTER2021,
    #     ( SELECT COUNT(ID), 
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07` 
    #     WHERE CLUSTER_2021 != CLUSTER_2020
    #     ) AS Percent
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY EMPRESA_J__NIOR
    #   ORDER BY CLUSTER2021 DESC
# In pandas:
NoJEProgressedCluster = resultsQuery1[resultsQuery1['CLUSTER2020'] != resultsQuery1['CLUSTER2021']]
display(NoJEProgressedCluster.count()/resultsQuery1.count())

ID             0.360000
CLUSTER2020    0.333333
CLUSTER2021    0.346939
dtype: float64


If the ID is taken into account, because it is the only value that can't be *null*, this means that only 0.36, or **36%**, of Junior Enterprises were **able to Progress in Cluster Classification**

Taking a Look at Cluster Proportion

In [22]:
# Cluster Proportion
# QUERY 2
#   SELECT CLUSTER_2020, 
    #   count(CLUSTER_2021) as Cluster2021_count,
    #   count(CLUSTER_2021) * 100.0 / (
    #       select count(*) from `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) 
    #       AS Cluster_percent2021
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   group by CLUSTER_2020
resultsQuery2 = pd.read_csv('results-query2.csv')
display(resultsQuery2)

,CLUSTER_2020,Cluster2021_count,Cluster_percent2021
0,1.0,169,37.977528
1,4.0,81,18.202247
2,NaN,3,0.674157
3,2.0,27,6.067416
4,5.0,22,4.943820
5,3.0,106,23.820225


From here it can be understood that most Junior Enterprises under NuSC's region are from Cluster 1, almost half of them are from the sum of 1 and 2.

With the 36% progress rate from 2020 to 2021 saw earlier, some questions and implications start to be seen: 
1. Is NuSC taking care of the Junior Enterprises from lower levels?
2. How are the other indicatiors in this manner?
2. Why aren't they progressing in Cluster Classification?

## 1. Cluster 1 & 2 Performance Analysis

To understand if Nucleo São Carlos is taking care of Junior Enterprises from Lower Clusters, being those from 1 and 2, it's important to understand what they represent among all 50 Junior Enterprises in the region (registered in the DB)

In [23]:
# CLUSTER 1 AND 2 PROPORTION
# QUERY 3
# SELECT DISTINCT EMPRESA_J__NIOR, (
    #   SELECT COUNT(DISTINCT ID)
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   ) AS JE_TOTAL, 
    #   (   SELECT COUNT(DISTINCT ID)
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   WHERE CLUSTER_2021 = 1 OR CLUSTER_2021 = 2) AS JE_C1N2, 
    #   (34 / 50)*100 AS PROPORTION
    #FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #GROUP BY EMPRESA_J__NIOR
resultsQuery3 = pd.read_csv('results-query3.csv')
display(resultsQuery3)

,EMPRESA_J__NIOR,JE_TOTAL,JE_C1N2,PROPORTION
0,ZOOT Jr.,50,34,68.0
1,Conecta Junior,50,34,68.0
2,FAECA JUNIOR,50,34,68.0
3,STARTUP! Engenharia,50,34,68.0
4,Sustec Jr.,50,34,68.0
5,Cultivar AgroJúnior,50,34,68.0
6,EJUR - Soluções Jurídicas,50,34,68.0
7,Orbe Consultoria Internacional,50,34,68.0
8,Uni-FACEF Júnior,50,34,68.0
9,Empresa Júnior das Ciências Unificadas - CiUni...,50,34,68.0


From here it's clear that most all junior enterprises registered in NuSC are from lower Clusters in 2021, 68%. But it's important to understand the progress rate from the Cluster 1 and 2 in the evaluation:

In [24]:
# Cluster Change from Junior Enterprises that are from lower Clusters (1,2)
# Query 4
# SELECT DISTINCT EMPRESA_J__NIOR, 
  #   CONCAT(CLUSTER_2020, " to ", 
  #   CLUSTER_2021) AS ClustProgress,
  #  FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`  
  #  WHERE CLUSTER_2021 != CLUSTER_2020 AND CLUSTER_2021 = 1 OR CLUSTER_2021 = 2
  #  GROUP BY EMPRESA_J__NIOR, CLUSTER_2020, CLUSTER_2021
resultsQuery4 = pd.read_csv('results-query4.csv')
display(resultsQuery4)

,EMPRESA_J__NIOR,ClustProgress
0,EJEAmb,2 to 2
1,Materiais Júnior,3 to 2
2,EJE - Empresa Júnior de Estatística,2 to 2
3,CATI Jr.,1 to 2
4,EMPsi Jr,2 to 2
5,EmpelTec Jr.,2 to 2
6,Biossistec Jr,1 to 2
7,Irhis Consultoria Júnior em Psicologia,2 to 2
8,Nutrirp Consultoria Júnior em Nutrição,4 to 2
9,Comunica Fono Jr.,1 to 2


It can be seen that only 4 Cluster Changes in the bottom part of Cluster Classification are in a positive climb, because all of other changes in Cluster 1 and 2 are from Junior Enterprises that dropped from higher rates.

## 2. Other Indicators from Junior Enterprises

Nothing can be implied without taking a look at the macro side of the enterprise and organization, so it's usefull to take into account other indicadors.

Revenue Analysis

In [25]:
# Junior Enterprises' Revenues
# QUERY 5
# SELECT DISTINCT EMPRESA_J__NIOR,CLUSTER_2020, CLUSTER_2021,
    #   SUM(CAST(FATURAMENTO AS INT64)) AS JERevenue,
    #   (SELECT SUM(CAST(FATURAMENTO AS INT64)) FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`)AS NUSCRevenue
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY EMPRESA_J__NIOR, CLUSTER_2020, CLUSTER_2021
    #   ORDER BY JERevenue DESC 
resultsQuery5 = pd.read_csv('results-query5.csv')
display(resultsQuery5)

,EMPRESA_J__NIOR,CLUSTER_2020,CLUSTER_2021,JERevenue,NUSCRevenue
0,EESC jr.,5.0,5.0,25303469,112158422
1,Júnior FEA-RP,4.0,5.0,13300500,112158422
2,Qualimentos Júnior,3.0,5.0,12546088,112158422
3,ICMC Júnior,4.0,4.0,11245500,112158422
4,Produção Jr,4.0,5.0,6848064,112158422
5,Engrenar Jr,3.0,4.0,6406300,112158422
6,EQ Júnior - Consultoria e Projetos,3.0,3.0,4852069,112158422
7,EJUR - Soluções Jurídicas,4.0,4.0,2781000,112158422
8,Química Jr,2.0,3.0,2664500,112158422
9,Paulista Jr. Projetos & Consultoria,1.0,4.0,2307000,112158422


Revenue by Cluster

In [13]:
# Revenues by Cluster
# QUERY 6
# SELECT DISTINCT CLUSTER_2021, 
    #   SUM(CAST(FATURAMENTO AS INT64)) AS ClusterRevenue,
    #   (SELECT SUM(CAST(FATURAMENTO AS INT64)) 
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) AS NUSCRevenue,
    #   (ClusterRevenue / NUSCRevenue)*100 AS Proportion,
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY CLUSTER_2021
    #   ORDER BY ClusterRevenue DESC 
resultsQuery6 = pd.read_csv('results-query6.csv')
display(resultsQuery6)

,CLUSTER_2021,ClusterRevenue,NUSCRevenue,Proportion
0,5.0,57998121,112158422,51.710892
1,4.0,24647300,112158422,21.975434
2,3.0,12458970,112158422,11.108368
3,2.0,10607819,112158422,9.457889
4,1.0,6193712,112158422,5.522289
5,NaN,252500,112158422,0.225128


So here it can be seen that only about 16% of total NUSC' revenue came from Cluster 1 and 2. While Cluster 5 supports more than half, even if they represent only about 5% of the total number of Junior Enterprises, as showed before.

Projects Analysis

In [14]:
# Projects by Cluster
# QUERY 7
# SELECT DISTINCT CLUSTER_2021, 
    #   SUM(CAST(PROJETOS AS INT64)) AS ClusterProjects,
    #   (SELECT SUM(CAST(PROJETOS AS INT64)) 
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) AS NUSCProjects,
    #   (ClusterProjects / NUSCProjects)*100 AS Proportion,
    #   AVG(TEMPO_M__DIO_POR_PROJETO__DIAS_) AS AVG_PROJ_TIME_IN_DAYS,
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY CLUSTER_2021
    #   ORDER BY ClusterProjects DESC 
resultsQuery7 = pd.read_csv('results-query7.csv')
display(resultsQuery7) 

,CLUSTER_2021,ClusterProjects,NUSCProjects,Proportion,AVG_PROJ_TIME_IN_DAYS
0,5.0,305,858,35.547786,32.470337
1,1.0,170,858,19.813520,15.835077
2,3.0,124,858,14.452214,37.612857
3,2.0,121,858,14.102564,22.393714
4,4.0,101,858,11.771562,52.654286
5,NaN,37,858,4.312354,2.410000


Here can be seen that the major revenue coming from Cluster 5 can be because it is the Cluster that executes projects the most. But the second one already is Cluster 1, which was the one that least contributes in revenue.

Aside from that, it is also important to understand how the average project execution time affects the sales process. Cluster 1 has the smallest average time, and Cluster 2 follows the previous one.

# So...Why aren't they progressing in Cluster Classification?

From this analysis some hypotesis can be taken into account. But what are the major metrics?

In [18]:
# Major Table with previous information and Created Metrics
# Query 8
# SELECT DISTINCT CLUSTER_2021, 
    #   SUM(CAST(PROJETOS AS INT64)) AS ClusterProjects,
    #   (SELECT SUM(CAST(PROJETOS AS INT64)) 
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) AS NUSCProjects,
    #   AVG(TEMPO_M__DIO_POR_PROJETO__DIAS_) AS AVG_PROJ_TIME_IN_DAYS, 
    #   SUM(CAST(FATURAMENTO AS INT64)) AS ClusterRevenue,
    #     (SELECT SUM(CAST(FATURAMENTO AS INT64))
    #     FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`) AS NUSCRevenue,
    #   ClusterRevenue/ClusterProjects AS Project_Avg_Ticket_byCluster
    #   FROM `dataset-orbe.IRHistoricalAnalysis.NUSC_2021_07`
    #   GROUP BY CLUSTER_2021
    #   ORDER BY ClusterRevenue DESC 
resultsQuery8 = pd.read_csv('results-query8.csv')
display(resultsQuery8)

,CLUSTER_2021,ClusterProjects,NUSCProjects,AVG_PROJ_TIME_IN_DAYS,ClusterRevenue,NUSCRevenue,Project_Avg_Ticket_byCluster
0,5.0,305,858,32.470337,57998121,112158422,190157.773800
1,4.0,101,858,52.654286,24647300,112158422,244032.673300
2,3.0,124,858,37.612857,12458970,112158422,100475.564500
3,2.0,121,858,22.393714,10607819,112158422,87667.925620
4,1.0,170,858,15.835077,6193712,112158422,36433.600000
5,NaN,37,858,2.410000,252500,112158422,6824.324324


Some hipotesys can be driven out of this analysis:
1. There is a major difference between a Junior Enterprise from higher Cluster and bottom Cluster, and that can impact in NUSC's mission to garantee an "entrepreneurial experience". As an example, if someone is from a Junior Enterprise from Cluster 1: its representation in revenue for its region is **just by 5%**, it is in the **category that least executes projects**, and in comparison with Cluster 5, **its average sales ticket is basically 5 times lower** (aprox. 19.15%). So this also means that this person isn't getting the experience a Junior Enterprise has to give him/her.
2. Also, when seing the average time to execute a project, the bottom Clusters also suffer, because, some metrics may indicate a problem in sales. In some common sense something with a lower price and a lower delivery period is easier to sell, if it maintains its quality. But, as seen, comparing the extremes (Cluster 1 and 5), the fist one has almost half of the average project execution time of the other, but their contributions to the total revenue have a difference of 10 times. This may indicate that the problem with bottom cluster Junior Enterprises is their sales process, their pitch or their sales structure in general. And it means that this is where NUSC would have to focus on to give them more experience.